<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Data Preparation
Resave data as a set of tiff files in order to match conventions expected by TracX.

In [26]:
import os
import sys

import numpy as np
import tensorflow as tf
from tifffile import imwrite

from deepcell.applications import NuclearSegmentation
from deepcell.datasets import DynamicNuclearNetTracking

sys.path.append('..')
import utils

In [27]:
data_dir = 'data'
ctc_dc_dir = os.path.join(data_dir, 'CTC', 'seg-dc')
ctc_gt_dir = os.path.join(data_dir, 'CTC', 'seg-gt')
raw_dir = os.path.join(data_dir, 'raw')
seg_gt_dir = os.path.join(data_dir, 'seg-gt')
seg_dc_dir = os.path.join(data_dir, 'seg-dc')

for d in [raw_dir, seg_gt_dir, seg_dc_dir, ctc_dc_dir, ctc_gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [28]:
# Load test data
dnn = DynamicNuclearNetTracking(version='1.1')
X, y, lineages = dnn.load_data(split='test')
data = {
    'X': X,
    'y': y,
    'lineages': lineages
}

INFO:root:Checking for cached data
INFO:root:Checking DynamicNuclearNet-tracking-v1_1.zip against provided file_hash...
INFO:root:DynamicNuclearNet-tracking-v1_1.zip with hash 4090ccd3b76c27f001f798c8187c5f3e already available.
INFO:root:Extracting /Users/morganschwartz/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip
INFO:root:Successfully extracted /Users/morganschwartz/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip into /Users/morganschwartz/.deepcell/datasets


Load the DeepCell nuclear segmentation model to test the algorithm on predicted instead of ground truth segmentations

In [29]:
app = NuclearSegmentation.from_version(version='1.1')

INFO:root:Checking for cached data
INFO:root:Checking NuclearSegmentation-8.tar.gz against provided file_hash...
INFO:root:NuclearSegmentation-8.tar.gz with hash 507be21f0e34e59adae689f58cc03ccb already available.
INFO:root:Extracting /Users/morganschwartz/.deepcell/models/NuclearSegmentation-8.tar.gz
INFO:root:Successfully extracted /Users/morganschwartz/.deepcell/models/NuclearSegmentation-8.tar.gz into /Users/morganschwartz/.deepcell/models


In [30]:
# ChannelName_positionXXYYZZZ_timeTTTT.tif
name_template = '{}_position{:02}{:02}000_time{:05}.tif'

In [31]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    seg_gt_subdir = os.path.join(seg_gt_dir, '{:03}'.format(batch_no + 1))
    seg_dc_subdir = os.path.join(seg_dc_dir, '{:03}'.format(batch_no + 1))

    # Create directories if needed
    for d in (raw_subdir, seg_gt_subdir, seg_dc_subdir):
        if not os.path.exists(d):
            os.makedirs(d)

    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]

    # Generate deepcell predictions
    y_pred = app.predict(X)

    # Save GT in CTC format
    for d in [ctc_dc_dir, ctc_gt_dir]:
        utils.save_ctc_gt(d, batch_no + 1, y[good_frames], lineage)

    # Position info for naming convention
    x_pos = batch_no + 1
    y_pos = 1

    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(X.shape[0]):
        name_raw = os.path.join(raw_subdir, name_template.format('nuclear', x_pos, y_pos, i+1))
        name_gt_mask = os.path.join(seg_gt_subdir, name_template.format('mask-gt', x_pos, y_pos, i+1))
        name_dc_mask = os.path.join(seg_dc_subdir, name_template.format('mask-dc', x_pos, y_pos, i+1))

        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_gt_mask, y[i, ..., channel])
        imwrite(name_dc_mask, y_pred[i, ..., channel])

INFO:root:Converting image dtype to float
2024-07-05 18:21:21.922759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-07-05 18:22:10.776814: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float
INFO:root:Converting image dtype to float


# 2. Tracking
1. Open Matlab > 2020 and open the TracX folder
2. From the Matlab terminal, run the following 
`addpath(genpath('tracx'));`
3. Open the `run_tracx.m` script in matlab and press run.

# 3. Evaluate

In [14]:
import glob
import os
import subprocess

import numpy as np
import pandas as pd
from tifffile import imread, imwrite

from deepcell_tracking.isbi_utils import load_tiffs
from deepcell_tracking.metrics import TrackingMetrics

In [20]:
data_dir = 'data'
ctc_dir = os.path.join(data_dir, 'CTC')
ctc_dc_dir = os.path.join(ctc_dir, 'seg-dc')
ctc_gt_dir = os.path.join(ctc_dir, 'seg-gt')
seg_gt_dir = os.path.join(data_dir, 'seg-gt')
seg_dc_dir = os.path.join(data_dir, 'seg-dc')

ids = os.listdir(seg_gt_dir)

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Mac' # or 'Mac' or 'Win'
num_digits = '3'

In [16]:
def create_new_lineage(y):
    """Create a blank lineage dict for ids that have already been
    linked via IOU. Link only based on overlap,
    so there are no divisions/daughters/parents/deaths
    Args:
        y: (np.array) label image stack.
    Returns:
        dict: a nested dict (lineage for .trk)
    """
    new_lineage = {}
    for i, frame in enumerate(y):
        # Add to frames field if ID exists
        cells_in_frame = np.unique(frame)
        cells_in_frame = np.delete(
            cells_in_frame, np.where(cells_in_frame == 0)
        )
        cells_in_frame = list(cells_in_frame)

        for cell in cells_in_frame:
            cell = int(cell)
            if cell in new_lineage:
                new_lineage[cell]["frames"].append(i)

            # Or create a new dict because its a new cell
            else:
                new_lineage[cell] = {
                    "label": cell,
                    "frames": [i],
                    "daughters": [],
                    "capped": False,
                    "frame_div": None,
                    "parent": None,
                }

    return new_lineage

Load each movie, relabel the mask to match the new tracking ids and create the lineage information

In [32]:
for res_dir in [seg_gt_dir, seg_dc_dir]:
    for i in ids:
        print(res_dir, i)
        # Find results path
        cc_path = glob.glob(os.path.join(res_dir, i, '*CellCycleResults*'))[0]
        t_path = glob.glob(os.path.join(res_dir, i, '*TrackingResults*'))[0]

        # Load TracX results
        cc_res = pd.read_csv(cc_path, sep='\t')
        t_res = pd.read_csv(t_path, sep='\t')

        y_old = load_tiffs(os.path.join(res_dir, i))

        # Create a new y array with update cell ids
        y_new = np.zeros_like(y_old)
        for _, r in t_res.iterrows():
            id_new = int(r['track_index'])
            id_old = int(r['cell_index'])
            t = int(r['cell_frame']) - 1 # Time indexed starting at 1 in TracX
            y_new[t][y_old[t] == id_old] = id_new

        # Create lineage
        lineage = create_new_lineage(y_new)

        # Assign parents and daughters
        for _, r in cc_res.iterrows():
            tid = r['track']
            # Check if a parent should be added
            if r['parent'] != 0:
                lineage[tid]['parent'] = int(r['parent'])

            # Check of daughter should be added
            if not np.isnan(r['daughter']):
                lineage[tid]['daughters'].append(int(r['daughter']))
                lineage[tid]['frame_div'] = max(lineage[tid]['frames'])

        # Correct discontiguous tracks, which are not allowed by CTC
        y_new, lineage = utils.convert_to_contiguous(y_new, lineage)

        # Save results in CTC format
        utils.save_ctc_res(os.path.join(ctc_dir, os.path.basename(res_dir)), int(i), y_new, lineage)

data/seg-dc 012
data/seg-dc 007
data/seg-dc 009
data/seg-dc 008
data/seg-dc 001
data/seg-dc 006
data/seg-dc 011
data/seg-dc 010
data/seg-dc 003
data/seg-dc 004
data/seg-dc 005
data/seg-dc 002


In [33]:
benchmarks = []

for results_dir, s in zip([ctc_gt_dir, ctc_dc_dir], ['GT', 'Deepcell']):
    for data_id in ids:
        print(data_id)
        results = {
            'model': f'TracX - {s}',
            'data_id': data_id
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)

        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout

            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))

        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

012
6_17 out degree = 1, daughters mismatch.
missed node 8_18 division completely
missed node 10_19 division completely
missed node 15_17 division completely
missed node 17_6 division completely
19_20 out degree = 1, daughters mismatch.
missed node 22_5 division completely
missed node 29_17 division completely
59_10 out degree = 1, daughters mismatch.
missed node 60_34 division completely
missed node 76_7 division completely
missed node 77_16 division completely
missed node 79_9 division completely
80_9 out degree = 1, daughters mismatch.
82_17 out degree = 1, daughters mismatch.
missed node 83_18 division completely
corrected division 10_19 as a frameshift division not an error
corrected division 60_34 as a frameshift division not an error
corrected division 76_7 as a frameshift division not an error
007
missed node 6_16 division completely
11_68 out degree = 1, daughters mismatch.
missed node 13_10 division completely
missed node 23_66 division completely
missed node 24_11 division c